In [5]:
import numpy as np
import cv2 as cv
import sys

def construct_yolo_v3():
    f=open('coco_names.txt', 'r')
    class_names=[line.strip() for line in f.readlines()]

    model=cv.dnn.readNet('yolov3.weights','yolov3.cfg')
    layer_names=model.getLayerNames()
    out_layers=[layer_names[i-1] for i in model.getUnconnectedOutLayers()]
    
    return model,out_layers,class_names

def yolo_detect(img,yolo_model,out_layers):
    height,width=img.shape[0],img.shape[1]
    test_img=cv.dnn.blobFromImage(img,1.0/256,(448,448),(0,0,0),swapRB=True)
    
    yolo_model.setInput(test_img)
    output3=yolo_model.forward(out_layers)
    
    box,conf,id=[],[],[]		# 박스, 신뢰도, 부류 번호
    for output in output3:
        for vec85 in output:
            scores=vec85[5:]
            class_id=np.argmax(scores)
            confidence=scores[class_id]
            if confidence>0.5:	# 신뢰도가 50% 이상인 경우만 취함
                centerx,centery=int(vec85[0]*width),int(vec85[1]*height)
                w,h=int(vec85[2]*width),int(vec85[3]*height)
                x,y=int(centerx-w/2),int(centery-h/2)
                box.append([x,y,x+w,y+h])
                conf.append(float(confidence))
                id.append(class_id)
            
    ind=cv.dnn.NMSBoxes(box,conf,0.5,0.4)
    objects=[box[i]+[conf[i]]+[id[i]] for i in range(len(box)) if i in ind]
    return objects

model,out_layers,class_names=construct_yolo_v3()	# YOLO 모델 생성
colors=np.random.uniform(0,255,size=(100,3))		# 100개 색으로 트랙 구분

from Sort import Sort

sort=Sort()

cap=cv.VideoCapture(0,cv.CAP_DSHOW)
if not cap.isOpened(): sys.exit('카메라 연결 실패')

while True:
    ret,frame=cap.read()
    if not ret: sys.exit('프레임 획득에 실패하여 루프를 나갑니다.')
        
    res=yolo_detect(frame,model,out_layers)   
    persons=[res[i] for i in range(len(res)) if res[i][5]==0] # 부류 0은 사람

    if len(persons)==0: 
        tracks=sort.update()
    else:
        tracks=sort.update(np.array(persons))
    
    for i in range(len(tracks)):
        x1,y1,x2,y2,track_id=tracks[i].astype(int)
        cv.rectangle(frame,(x1,y1),(x2,y2),colors[track_id],2)
        cv.putText(frame,str(track_id),(x1+10,y1+40),cv.FONT_HERSHEY_PLAIN,3,colors[track_id],2)            
    
    cv.imshow('Person tracking by SORT',frame)
    
    key=cv.waitKey(1) 
    if key==ord('q'): break 
    
cap.release()		# 카메라와 연결을 끊음
cv.destroyAllWindows()

Process

1. Yolo나 RCNN의 계열의 객체 인식 모델로 정지된 사진 (프레임)에 존재하는 Object를 탐지
- 이때, Confidence에 따라 신뢰도가 높은 BBOX를 구하고 BBOX이외의 배경화면에 대해서는 날리고, BBox를 B_detection 정보에 담는다.

2. 특징 추출 단계
- 이전 순간 t-1에서 결정한 목표물의 위치 정보와 이동 이력 정보를 사용
- B(x,y,s,r,x;,y;,s;) x,y,s는 목표물의 중심위치와 크기이고, r은 높이와 너비의 비율, x;y;s;는 각각 목표물이 이전에 이동했던 정보를 누적한 이력 정보이다.
t-1의 목표물 정보로 t순간을 예측하고 t순간의 BBOX를 B_pred에 담는다.

3. B_detection에 담긴 정보와 B_pred에 담긴 BBOX 정보의 IOU를 계산하고 1에서 IOU를 빼 거리로 변환한다.
둘 간의 갯수가 다른 경우 부족한 쪽의 가상 박스를 채워 정방 행렬로 만든다. 가상 박스의 거리는 높게 설정하여 매칭이 일어나지 않게 설정

4. 거리행렬을 이용하여 Hungarian Algorithm (최적의 저비용 매칭 알고리즘)을 이용하여 최적의 BBOX의 매칭 쌍을 찾는다.

4단계를 거치면 Video Tracking이 완성된다.

그러나 후처리과정이 필요하다.
바로 2. 에 있는 B_pred에 대한 정보를 갱신하는 일이다. 매칭이 되었던 BBOX의 x,y,s,r을 대치하고, 
이동 이력 정보에 해당했던, x;y;s;는 Kalman Filter를 사용하여 갱신한다. 잡음과 변형이 심한 시계열 데이터에서 이전 샘플의 분포를 감안하여 현재 측정치를 보완하는 기법이다.

이 칼만 필터를 이용하여 필수적으로 발생하는 잡음을 잡아주어 측정치를 보완한 뒤에 이동 이력 정보에 추가한다.
